In [0]:
#https://www.kaggle.com/c/msk-redefining-cancer-treatment

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
import warnings
from sklearn import ensemble

warnings.filterwarnings("ignore")
from sklearn.preprocessing import normalize

from scipy.sparse import hstack
from sklearn.naive_bayes import MultinomialNB

In [146]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
data_variants = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/training/training_variants')
data_text =pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/training/training_text',sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [148]:
data_variants.head(2)

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2


In [149]:
data_text.head(2)

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...


In [150]:
data_variants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 4 columns):
ID           3321 non-null int64
Gene         3321 non-null object
Variation    3321 non-null object
Class        3321 non-null int64
dtypes: int64(2), object(2)
memory usage: 103.9+ KB


In [151]:

data_variants.describe()

,ID,Class
count,3321.000000,3321.000000
mean,1660.000000,4.365854
std,958.834449,2.309781
min,0.000000,1.000000
25%,830.000000,2.000000
50%,1660.000000,4.000000
75%,2490.000000,7.000000
max,3320.000000,9.000000


In [152]:
data_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 2 columns):
ID      3321 non-null int64
TEXT    3316 non-null object
dtypes: int64(1), object(1)
memory usage: 52.0+ KB


In [153]:
data_variants.Class.unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [154]:
data_text.isnull().sum()

ID      0
TEXT    5
dtype: int64

In [155]:
data_text.isnull().any(axis=1)

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3291    False
3292    False
3293    False
3294    False
3295    False
3296    False
3297    False
3298    False
3299    False
3300    False
3301    False
3302    False
3303    False
3304    False
3305    False
3306    False
3307    False
3308    False
3309    False
3310    False
3311    False
3312    False
3313    False
3314    False
3315    False
3316    False
3317    False
3318    False
3319    False
3320    False
Length: 3321, dtype: bool

In [156]:
data_text[data_text.isnull().any(axis=1)]

,ID,TEXT
1109,1109,NaN
1277,1277,NaN
1407,1407,NaN
1639,1639,NaN
2755,2755,NaN


In [157]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import re
def data_text_preprocess(total_text, ind, col):
    if type(total_text) is not int:
        string = ""
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        total_text = re.sub('\s+',' ', str(total_text))
        total_text = total_text.lower()
        
        for word in total_text.split():
            if not word in stop_words:
                string += word + " "
        
        data_text[col][ind] = string

In [159]:
for index, row in data_text.iterrows():
    if type(row['TEXT']) is str:
        data_text_preprocess(row['TEXT'], index, 'TEXT')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [160]:
result = pd.merge(data_variants, data_text,on='ID', how='left')
result.head()

,ID,Gene,Variation,Class,TEXT
0,0,FAM58A,Truncating Mutations,1,cyclin dependent kinases cdks regulate variety...
1,1,CBL,W802*,2,abstract background non small cell lung cancer...
2,2,CBL,Q249E,2,abstract background non small cell lung cancer...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutations monomeric casitas b lineag...


In [161]:
result.loc[result['TEXT'].isnull()]

,ID,Gene,Variation,Class,TEXT
1109,1109,FANCA,S1088F,1,NaN
1277,1277,ARID5B,Truncating Mutations,1,NaN
1407,1407,FGFR3,K508M,6,NaN
1639,1639,FLT1,Amplification,6,NaN
2755,2755,BRAF,G596C,7,NaN


In [0]:
result.dropna(inplace=True)

In [163]:
result.loc[result['TEXT'].isnull()]

,ID,Gene,Variation,Class,TEXT


In [164]:
data_variants.Class.unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
y_true = result['Class'].values
result.Gene      = result.Gene.str.replace('\s+', '_')
result.Variation = result.Variation.str.replace('\s+', '_')
from sklearn.model_selection import train_test_split
X_train, test_df, y_train, y_test = train_test_split(result, y_true, stratify=y_true, test_size=0.2)
train_df, cv_df, y_train, y_cv = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)

In [166]:
print('Number of data points in train data:', train_df.shape[0])
print('Number of data points in test data:', test_df.shape[0])
print('Number of data points in cross validation data:', cv_df.shape[0])

Number of data points in train data: 2121
Number of data points in test data: 664
Number of data points in cross validation data: 531


In [167]:
train_df['Class'].value_counts().sortlevel()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: sortlevel is deprecated, use sort_index(level=...)
  """Entry point for launching an IPython kernel.


1    362
2    289
3     57
4    439
5    155
6    174
7    609
8     12
9     24
Name: Class, dtype: int64

In [168]:
test_df['Class'].value_counts().sortlevel()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: sortlevel is deprecated, use sort_index(level=...)
  """Entry point for launching an IPython kernel.


1    113
2     91
3     18
4    137
5     48
6     55
7    191
8      4
9      7
Name: Class, dtype: int64

In [169]:
cv_df['Class'].value_counts().sortlevel()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: sortlevel is deprecated, use sort_index(level=...)
  """Entry point for launching an IPython kernel.


1     91
2     72
3     14
4    110
5     39
6     44
7    152
8      3
9      6
Name: Class, dtype: int64

In [0]:
##building a complete random model

In [0]:
from sklearn.metrics.classification import accuracy_score, log_loss


In [0]:
import numpy as np
test_data_len = test_df.shape[0]
cv_data_len = cv_df.shape[0]

In [173]:
y_cv

array([4, 2, 4, 1, 7, 2, 1, 4, 7, 7, 7, 4, 1, 8, 7, 6, 6, 7, 2, 4, 6, 7,
       2, 4, 7, 4, 4, 1, 2, 6, 4, 7, 4, 4, 4, 4, 4, 1, 7, 5, 2, 2, 2, 6,
       7, 1, 2, 7, 7, 2, 7, 3, 6, 1, 2, 4, 7, 1, 5, 7, 7, 2, 1, 6, 3, 2,
       1, 4, 7, 3, 4, 4, 1, 5, 6, 7, 7, 7, 2, 5, 4, 7, 7, 6, 7, 7, 2, 4,
       7, 5, 4, 2, 2, 1, 7, 4, 4, 4, 7, 7, 7, 4, 4, 2, 2, 1, 7, 7, 1, 2,
       7, 7, 1, 1, 4, 5, 4, 4, 1, 7, 1, 6, 4, 7, 6, 1, 4, 7, 6, 4, 2, 5,
       4, 4, 5, 5, 1, 4, 4, 4, 7, 7, 1, 4, 6, 4, 6, 5, 7, 4, 6, 2, 3, 1,
       1, 7, 6, 4, 1, 6, 2, 2, 7, 5, 2, 7, 1, 1, 4, 7, 8, 7, 7, 7, 7, 2,
       9, 1, 7, 2, 2, 4, 2, 7, 3, 7, 7, 1, 1, 6, 7, 2, 1, 5, 7, 4, 5, 4,
       4, 5, 1, 1, 7, 4, 6, 7, 6, 4, 6, 2, 7, 1, 7, 4, 7, 4, 7, 9, 2, 6,
       1, 7, 1, 1, 4, 6, 7, 2, 5, 7, 7, 7, 1, 4, 7, 9, 4, 7, 7, 6, 1, 2,
       7, 4, 4, 1, 1, 2, 7, 6, 1, 2, 2, 7, 1, 1, 2, 1, 6, 4, 4, 4, 7, 1,
       6, 1, 7, 5, 4, 7, 1, 5, 2, 1, 2, 1, 5, 6, 1, 2, 3, 4, 2, 6, 5, 5,
       4, 4, 7, 2, 1, 4, 5, 7, 7, 3, 2, 7, 6, 8, 7,

In [0]:
cv_predicted_y = np.zeros((cv_data_len,9))
for i in range(cv_data_len):
    rand_probs = np.random.rand(1,9)
    cv_predicted_y[i] = ((rand_probs/sum(sum(rand_probs)))[0])


In [175]:
cv_predicted_y.shape

(531, 9)

In [176]:
y_cv.shape

(531,)

In [177]:
print("Log loss on Cross Validation Data using Random Model",log_loss(y_cv,cv_predicted_y, eps=1e-15))

Log loss on Cross Validation Data using Random Model 2.4899530870484083


In [178]:
# Test-Set error.

test_predicted_y = np.zeros((test_data_len,9))
for i in range(test_data_len):
    rand_probs = np.random.rand(1,9)
    test_predicted_y[i] = ((rand_probs/sum(sum(rand_probs)))[0])
print("Log loss on Test Data using Random Model",log_loss(y_test,test_predicted_y, eps=1e-15))


Log loss on Test Data using Random Model 2.443582141318987


In [0]:
# Lets get the index of max probablity
predicted_y =np.argmax(test_predicted_y, axis=1)

In [0]:
predicted_y = predicted_y + 1

In [0]:
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_test, predicted_y)

In [182]:
c

array([[15, 10, 12,  6, 12, 16, 16, 12, 14],
       [13,  7,  7, 11,  8, 14,  9, 11, 11],
       [ 2,  2,  3,  2,  2,  3,  2,  2,  0],
       [19, 18, 13, 15, 15, 14, 11, 15, 17],
       [ 4, 10,  9,  4,  3,  5,  3,  4,  6],
       [ 9, 10,  6,  7,  7,  3,  7,  4,  2],
       [18, 20, 16, 20, 19, 22, 29, 27, 20],
       [ 1,  0,  1,  0,  0,  0,  1,  0,  1],
       [ 1,  2,  0,  1,  2,  0,  0,  1,  0]])

In [183]:
unique_genes = train_df['Gene'].value_counts()
print('Number of Unique Genes :', unique_genes.shape[0])
print(unique_genes.head(10))

Number of Unique Genes : 228
BRCA1     174
TP53      106
EGFR       84
BRCA2      82
PTEN       82
KIT        62
BRAF       58
ERBB2      44
ALK        43
PDGFRA     39
Name: Gene, dtype: int64


In [0]:
# one-hot encoding of Gene feature.
gene_vectorizer = CountVectorizer()
train_gene_feature_onehotCoding = gene_vectorizer.fit_transform(train_df['Gene'])
test_gene_feature_onehotCoding = gene_vectorizer.transform(test_df['Gene'])
cv_gene_feature_onehotCoding = gene_vectorizer.transform(cv_df['Gene'])

In [185]:
train_gene_feature_onehotCoding

<2121x227 sparse matrix of type '<class 'numpy.int64'>'
	with 2121 stored elements in Compressed Sparse Row format>

In [186]:
gene_vectorizer.get_feature_names()

['abl1',
 'acvr1',
 'ago2',
 'akt1',
 'akt2',
 'akt3',
 'alk',
 'apc',
 'ar',
 'araf',
 'arid1a',
 'arid1b',
 'arid2',
 'asxl1',
 'asxl2',
 'atm',
 'atr',
 'atrx',
 'aurka',
 'aurkb',
 'axl',
 'b2m',
 'bap1',
 'bcl10',
 'bcl2l11',
 'bcor',
 'braf',
 'brca1',
 'brca2',
 'brd4',
 'brip1',
 'card11',
 'carm1',
 'casp8',
 'cbl',
 'ccnd1',
 'ccnd2',
 'ccnd3',
 'ccne1',
 'cdh1',
 'cdk12',
 'cdk4',
 'cdk6',
 'cdkn1a',
 'cdkn1b',
 'cdkn2a',
 'cdkn2b',
 'cebpa',
 'chek2',
 'cic',
 'crebbp',
 'ctcf',
 'ctla4',
 'ctnnb1',
 'ddr2',
 'dicer1',
 'dnmt3a',
 'dnmt3b',
 'egfr',
 'elf3',
 'ep300',
 'epas1',
 'erbb2',
 'erbb3',
 'erbb4',
 'ercc2',
 'ercc3',
 'ercc4',
 'erg',
 'esr1',
 'etv1',
 'etv6',
 'ewsr1',
 'ezh2',
 'fam58a',
 'fanca',
 'fancc',
 'fat1',
 'fbxw7',
 'fgfr1',
 'fgfr2',
 'fgfr3',
 'fgfr4',
 'flt3',
 'foxa1',
 'foxl2',
 'foxo1',
 'foxp1',
 'gata3',
 'gna11',
 'gnas',
 'h3f3a',
 'hist1h1c',
 'hla',
 'hnf1a',
 'hras',
 'idh1',
 'idh2',
 'igf1r',
 'ikzf1',
 'il7r',
 'jak1',
 'jak2',
 'jun'

In [187]:
train_gene_feature_onehotCoding.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [0]:
value_count = train_df['Gene'].value_counts()

In [189]:
value_count.items()

In [190]:
for i, denominator in value_count.items():
  print(f'{i},{denominator}')

BRCA1,174
TP53,106
EGFR,84
BRCA2,82
PTEN,82
KIT,62
BRAF,58
ERBB2,44
ALK,43
PDGFRA,39
FLT3,37
PIK3CA,34
VHL,32
TSC2,30
MTOR,29
CDKN2A,29
FGFR2,29
FGFR3,27
KRAS,27
MET,24
MAP2K1,23
RET,23
MLH1,22
JAK2,22
SMAD4,21
AKT1,19
PTPN11,19
NOTCH1,18
CTNNB1,17
ABL1,17
PDGFRB,16
SPOP,16
PTPRT,16
AR,15
RUNX1,15
FBXW7,14
SMO,14
NFE2L2,14
PIK3R1,14
SMAD2,14
ROS1,14
ERBB4,14
MSH2,14
CBL,13
HRAS,13
PMS2,12
SMAD3,12
TSC1,12
JAK1,12
KEAP1,11
SF3B1,11
MSH6,11
TET2,10
ERCC2,10
CCND1,10
ESR1,10
IDH1,9
PIK3CB,9
NRAS,9
NF2,9
NTRK1,9
STK11,9
PPP2R1A,9
NF1,8
CDH1,7
MAP2K4,7
TERT,7
CDK12,7
CHEK2,7
PIK3R2,7
CARD11,7
MAP2K2,7
CREBBP,6
RB1,6
RAC1,6
RHOA,6
FGFR1,6
EWSR1,6
POLE,6
CCND3,6
SOX9,5
DDR2,5
DICER1,5
FANCA,5
CDKN2B,5
MEF2B,5
EPAS1,5
EP300,5
ERCC4,5
SMARCA4,5
AKT2,5
RAF1,5
MYC,5
STAT3,4
BAP1,4
MED12,4
RIT1,4
SOS1,4
PIM1,4
KDR,4
CIC,4
B2M,4
MAP3K1,4
PRDM1,4
AGO2,4
ERG,4
EZH2,4
ELF3,4
CASP8,4
FOXA1,4
DNMT3A,3
U2AF1,3
RASA1,3
ARAF,3
GNAS,3
NKX2-1,3
BRIP1,3
TGFBR1,3
PIK3CD,3
TMPRSS2,3
KMT2C,3
BCOR,3
ATM,3
CDKN1A,

In [191]:
train_df.loc[(train_df['Class']==1) & (train_df['Gene']=='BRCA1')]

,ID,Gene,Variation,Class,TEXT
2523,2523,BRCA1,L1854P,1,genetic screening breast ovarian cancer suscep...
2421,2421,BRCA1,Truncating_Mutations,1,brca1 brca2 mutations estimated responsible gr...
2599,2599,BRCA1,M1689R,1,abstract brca1 gene individuals risk breast ov...
2577,2577,BRCA1,A1752V,1,genetic screening breast ovarian cancer suscep...
2664,2664,BRCA1,A1843P,1,abstract brca1 gene individuals risk breast ov...
2583,2583,BRCA1,E1660G,1,abstract brca1 gene individuals risk breast ov...
2475,2475,BRCA1,R1699L,1,abstract brca1 gene individuals risk breast ov...
2659,2659,BRCA1,A1708V,1,genetic screening breast ovarian cancer suscep...
2677,2677,BRCA1,A1843T,1,abstract brca1 gene individuals risk breast ov...
2455,2455,BRCA1,H1746N,1,abstract brca1 gene individuals risk breast ov...


In [0]:
cls_cnt = train_df.loc[(train_df['Class']==1) &(train_df['Gene']=='BRCA1')]

In [193]:
cls_cnt.shape

(44, 5)

In [194]:
(cls_cnt.shape[0] + 1*10)/ (169 + 90*1)

0.2084942084942085

In [0]:
# alpha : used for laplace smoothing

def get_gv_fea_dict(alpha, feature, df):

    value_count = train_df[feature].value_counts()
    
    # gv_dict : Gene Variation Dict, which contains the probability array for each gene/variation
    gv_dict = dict()
    
    # denominator will contain the number of time that particular feature occured in whole data
    for i, denominator in value_count.items():
        # vec will contain (p(yi==1/Gi) probability of gene/variation belongs to perticular class
        vec = []
        for k in range(1,10):


            cls_cnt = train_df.loc[(train_df['Class']==k) & (train_df[feature]==i)]
            
            # cls_cnt.shape[0](numerator) will contain the number of time that particular feature occured in whole data
            vec.append((cls_cnt.shape[0] + alpha*10)/ (denominator + 90*alpha))

        # we are adding the gene/variation to the dict as key and vec as value
        gv_dict[i]=vec
    return gv_dict

# Get Gene variation feature
def get_gv_feature(alpha, feature, df):

    gv_dict = get_gv_fea_dict(alpha, feature, df)
    # value_count is similar in get_gv_fea_dict
    value_count = train_df[feature].value_counts()
    
    # gv_fea: Gene_variation feature, it will contain the feature for each feature value in the data
    gv_fea = []
    # for every feature values in the given data frame we will check if it is there in the train data then we will add the feature to gv_fea
    # if not we will add [1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9] to gv_fea
    for index, row in df.iterrows():
        if row[feature] in dict(value_count).keys():
            gv_fea.append(gv_dict[row[feature]])
        else:
            gv_fea.append([1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9])
#             gv_fea.append([-1,-1,-1,-1,-1,-1,-1,-1,-1])
    return gv_fea

In [0]:

alpha = 1
train_gene_feature_responseCoding = np.array(get_gv_feature(alpha, "Gene", train_df))
test_gene_feature_responseCoding = np.array(get_gv_feature(alpha, "Gene", test_df))
cv_gene_feature_responseCoding = np.array(get_gv_feature(alpha, "Gene", cv_df))

In [197]:
train_gene_feature_responseCoding

array([[0.20454545, 0.03787879, 0.0719697 , ..., 0.03787879, 0.03787879,
        0.03787879],
       [0.09615385, 0.10576923, 0.09615385, ..., 0.22115385, 0.09615385,
        0.09615385],
       [0.19230769, 0.09615385, 0.09615385, ..., 0.09615385, 0.09615385,
        0.09615385],
       ...,
       [0.20454545, 0.03787879, 0.0719697 , ..., 0.03787879, 0.03787879,
        0.03787879],
       [0.13461538, 0.10576923, 0.10576923, ..., 0.09615385, 0.09615385,
        0.09615385],
       [0.10377358, 0.19811321, 0.09433962, ..., 0.11320755, 0.09433962,
        0.09433962]])

In [207]:
alpha = [10 ** x for x in range(-5, 1)]
cv_log_error_array=[]
for i in alpha:
    clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
    clf.fit(train_gene_feature_onehotCoding, y_train)
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_gene_feature_onehotCoding, y_train)
    predict_y = sig_clf.predict_proba(cv_gene_feature_onehotCoding)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))

For values of alpha =  1e-05 The log loss is: 1.4159278398072992
For values of alpha =  0.0001 The log loss is: 1.263137048151398
For values of alpha =  0.001 The log loss is: 1.2773543876351925
For values of alpha =  0.01 The log loss is: 1.3851347894369335
For values of alpha =  0.1 The log loss is: 1.4586695240349388
For values of alpha =  1 The log loss is: 1.4915383750899114


In [210]:
cv_log_error_array=[]
for i in alpha:
    clf = ensemble.GradientBoostingClassifier(n_estimators=100)
    clf.fit(train_gene_feature_onehotCoding, y_train)
    
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_gene_feature_onehotCoding, y_train)
    predict_y = sig_clf.predict_proba(cv_gene_feature_onehotCoding)
    
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))

For values of alpha =  1e-05 The log loss is: 1.3372963279418413
For values of alpha =  0.0001 The log loss is: 1.3368334205222492
For values of alpha =  0.001 The log loss is: 1.3375997362273906
For values of alpha =  0.01 The log loss is: 1.3374113958171083
For values of alpha =  0.1 The log loss is: 1.3370882298167999
For values of alpha =  1 The log loss is: 1.3370671870422606


In [0]:
variation_vectorizer = CountVectorizer()
train_variation_feature_onehotCoding = variation_vectorizer.fit_transform(train_df['Variation'])
test_variation_feature_onehotCoding = variation_vectorizer.transform(test_df['Variation'])
cv_variation_feature_onehotCoding = variation_vectorizer.transform(cv_df['Variation'])

In [212]:
alpha = [10 ** x for x in range(-5, 1)]
cv_log_error_array=[]
for i in alpha:
    clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
    clf.fit(train_variation_feature_onehotCoding, y_train)
    
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_variation_feature_onehotCoding, y_train)
    predict_y = sig_clf.predict_proba(cv_variation_feature_onehotCoding)
    
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))

For values of alpha =  1e-05 The log loss is: 1.7274846623664872
For values of alpha =  0.0001 The log loss is: 1.7017232686450012
For values of alpha =  0.001 The log loss is: 1.70092564067032
For values of alpha =  0.01 The log loss is: 1.709644264120436
For values of alpha =  0.1 The log loss is: 1.7213684810754908
For values of alpha =  1 The log loss is: 1.7223933727899408


In [0]:
text_vectorizer = CountVectorizer(min_df=3)
train_text_feature_onehotCoding = text_vectorizer.fit_transform(train_df['TEXT'])

In [0]:
train_text_feature_onehotCoding = normalize(train_text_feature_onehotCoding, axis=0)

test_text_feature_onehotCoding = text_vectorizer.transform(test_df['TEXT'])
test_text_feature_onehotCoding = normalize(test_text_feature_onehotCoding, axis=0)

cv_text_feature_onehotCoding = text_vectorizer.transform(cv_df['TEXT'])
cv_text_feature_onehotCoding = normalize(cv_text_feature_onehotCoding, axis=0)

In [217]:
cv_log_error_array=[]
for i in alpha:
    clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
    clf.fit(train_text_feature_onehotCoding, y_train)
    
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_text_feature_onehotCoding, y_train)
    predict_y = sig_clf.predict_proba(cv_text_feature_onehotCoding)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))


For values of alpha =  1e-05 The log loss is: 1.3613634131079944
For values of alpha =  0.0001 The log loss is: 1.3341559250299437
For values of alpha =  0.001 The log loss is: 1.2454251348125744
For values of alpha =  0.01 The log loss is: 1.2679353826367006
For values of alpha =  0.1 The log loss is: 1.4361720756470169
For values of alpha =  1 The log loss is: 1.6530623028319165


In [0]:
#MCombining all 3 features together¶

In [0]:
train_gene_var_onehotCoding = hstack((train_gene_feature_onehotCoding,train_variation_feature_onehotCoding))
test_gene_var_onehotCoding = hstack((test_gene_feature_onehotCoding,test_variation_feature_onehotCoding))
cv_gene_var_onehotCoding = hstack((cv_gene_feature_onehotCoding,cv_variation_feature_onehotCoding))

train_x_onehotCoding = hstack((train_gene_var_onehotCoding, train_text_feature_onehotCoding)).tocsr()
train_y = np.array(list(train_df['Class']))

test_x_onehotCoding = hstack((test_gene_var_onehotCoding, test_text_feature_onehotCoding)).tocsr()
test_y = np.array(list(test_df['Class']))

cv_x_onehotCoding = hstack((cv_gene_var_onehotCoding, cv_text_feature_onehotCoding)).tocsr()
cv_y = np.array(list(cv_df['Class']))





In [224]:
alpha = [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]
cv_log_error_array = []
for i in alpha:
    print("for alpha =", i)
    clf = MultinomialNB(alpha=i)
    clf.fit(train_x_onehotCoding, train_y)
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_x_onehotCoding, train_y)
    sig_clf_probs = sig_clf.predict_proba(cv_x_onehotCoding)
    cv_log_error_array.append(log_loss(cv_y, sig_clf_probs, labels=clf.classes_, eps=1e-15))
    print("Log Loss :",log_loss(cv_y, sig_clf_probs)) 

for alpha = 1e-05
Log Loss : 1.2906139995124812
for alpha = 0.0001
Log Loss : 1.2821296407318135
for alpha = 0.001
Log Loss : 1.2745228115674703
for alpha = 0.1
Log Loss : 1.259183812193307
for alpha = 1
Log Loss : 1.2761803579659372
for alpha = 10
Log Loss : 1.3657413830040381
for alpha = 100
Log Loss : 1.3295920710347737
for alpha = 1000
Log Loss : 1.306100279111112
